# Hacker News Post Analysis

## Introduction

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- id: The unique identifier from Hacker News for the post
- title: The title of the post
- url: The URL that the posts links to, if it the post has a URL
- num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- num_comments: The number of comments that were made on the post
- author: The username of the person who submitted the post
- created_at: The date and time at which the post was submitted

Here is an example column:

|id|title|	url	|num_points|num_comments|author	| created_at|
|-----|-----|----------|------------|-------|---------|
|12224879|Interactive Dynamic Video|http://www.interactivedynamicvideo.com/ |386	|52	|ne0phyte	|8/4/2016 

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:
- Ask HN: How to improve my personal website?
- Ask HN: Am I the only one outraged by Twitter shutting down share counts?
- Ask HN: Aby recent changes to CSS that broke mobile?

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:
-Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
-Show HN: Something pointless I made
-Show HN: Shanhu.io, a programming playground powered by e8vm

We'll compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?

## Miscellaneous Functions

In [7]:
### Nice Printing ###
def nice_print(list):
    for item in list:
        print(item)
        print("\n")

## Section 1: Importing and filtering the dataset

In [8]:
from csv import *
open_file=open('hacker_news.csv')
read_file=reader(open_file)
hn=list(read_file)
headers=hn[0] #HEADER
hn=hn[1:]       #ALL DATA
print(headers)
print('\n')
nice_print(hn[:5])
print("The length of the data set is:",len(hn))

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']


The length

## Filtering for Ask HN or Show HN:
### Using 'startswith( )' and 'lower( )'
Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [9]:
ask_posts=[]
show_posts=[]
other_posts=[]

for item in hn:
    title=item[1]
    title=title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(item)
    elif title.startswith('show hn'):
        show_posts.append(item)
    else:
        other_posts.append(item)
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))
sum_lists=len(ask_posts)+len(show_posts)+len(other_posts)
print("The sum of these sets is", sum_lists,"and it should be", len(hn))
print("\n")
print("The first 5 posts in ask_posts are:")
print("\n")
nice_print(ask_posts[:5])
print("The first 5 posts in show_posts are:")
print("\n")
nice_print(show_posts[:5])
print("The first 5 posts in other_posts are:")
print("\n")
nice_print(other_posts[:5])

1744
1162
17194
The sum of these sets is 20100 and it should be 20100


The first 5 posts in ask_posts are:


['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']


['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']


['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']


['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20']


['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']


The first 5 posts in show_posts are:


['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']


['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '74

## Section 2: Analysing the Ask and Show posts

Next, let's determine if ask posts or show posts receive more comments on average.

In [10]:
### Calculating total and avg comments for ASK posts ####
total_ask_comments=0
for item in ask_posts:
    n_comments=int(item[4])
    total_ask_comments+=n_comments
avg_ask_comments=round(total_ask_comments/len(ask_posts))
print("The total number of comments on Ask Posts was:",total_ask_comments)
print("The average number of comments on Ask Posts was:", avg_ask_comments)
print("\n")

### Calculating total and avg comments for SHOW posts ####
total_show_comments=0
for item in show_posts:
    n_comments=int(item[4])
    total_show_comments+=n_comments
avg_show_comments=round(total_show_comments/len(show_posts))
print("The total number of comments on Show Posts was:",total_show_comments)
print("The average number of comments on Show Posts was:", avg_show_comments)

The total number of comments on Ask Posts was: 24483
The average number of comments on Ask Posts was: 14


The total number of comments on Show Posts was: 11988
The average number of comments on Show Posts was: 10


### Show vs Ask comment rates
Ask posts get 40% more comments on average than show posts. They also appear about twice as frequently in this sample set.
At first glance this makes sense as the ask_posts are of course specifically asking users for comment. 

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

### How Time of posts effects comment rate
We'll use the following steps to perform this analysis:

1. Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

We'll use the datetime module to work with the data in the created_at column.

Recall that we can use the datetime.strptime() constructor to parse dates stored as strings and return datetime objects:

In [23]:
import datetime as dt
result_list=[]
for item in ask_posts:
    created_at=item[6]
    n_comments=int(item[4])
    result_list.append([created_at,n_comments])

counts_by_hour={} ### Tracks a freq table for number of posts per hour
comments_by_hour={} ### Tracks a for table for number of comments per hour
for item in result_list:
    hour=item[0]
    n_comments=item[1]
    hour = dt.datetime.strptime(hour, "%m/%d/%Y %H:%M")
    hour = dt.datetime.strftime(hour, "%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour]=1
        comments_by_hour[hour]=n_comments
    else:
        counts_by_hour[hour]+=1
        comments_by_hour[hour]+=n_comments
print("counts_by_hour")
print(counts_by_hour)
print("\n")
print("comments_by_hour")
print(comments_by_hour)


counts_by_hour
{'06': 44, '02': 58, '11': 58, '21': 109, '05': 46, '19': 110, '14': 107, '12': 73, '17': 100, '04': 47, '00': 55, '01': 60, '09': 45, '10': 59, '23': 68, '08': 48, '16': 108, '07': 34, '20': 80, '22': 71, '13': 85, '15': 116, '18': 109, '03': 54}


comments_by_hour
{'06': 397, '02': 1381, '11': 641, '21': 1745, '05': 464, '19': 1188, '14': 1416, '12': 687, '17': 1146, '04': 337, '00': 447, '01': 683, '09': 251, '10': 793, '23': 543, '08': 492, '16': 1814, '07': 267, '20': 1722, '22': 479, '13': 1253, '15': 4477, '18': 1439, '03': 421}


### Calculating Avergae CommentsPer Post for Each Hour
We'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [40]:
### Calculating the avergae number of comments per post per hour ###
avg_by_hour=[]
for item in counts_by_hour:
    hour=item
    n_posts=int(counts_by_hour[hour])
    n_comments=int(comments_by_hour[hour])
    if posts != 0:
        avg_val=round(n_comments/n_posts,2)
        avg_by_hour.append([hour,avg_val])
print(avg_by_hour)


[['06', 9.02], ['02', 23.81], ['11', 11.05], ['21', 16.01], ['05', 10.09], ['19', 10.8], ['14', 13.23], ['12', 9.41], ['17', 11.46], ['04', 7.17], ['00', 8.13], ['01', 11.38], ['09', 5.58], ['10', 13.44], ['23', 7.99], ['08', 10.25], ['16', 16.8], ['07', 7.85], ['20', 21.52], ['22', 6.75], ['13', 14.74], ['15', 38.59], ['18', 13.2], ['03', 7.8]]


### Sorting the Frequency Table 
Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [54]:
swap_avg_by_hour=[]
for item in avg_by_hour:
    swap_avg_by_hour.append([item[1],item[0]])

print(swap_avg_by_hour)
print("\n") # Sort with most comments per hour first 
print(sorted(swap_avg_by_hour, reverse=True))
print("\n") 


[[9.02, '06'], [23.81, '02'], [11.05, '11'], [16.01, '21'], [10.09, '05'], [10.8, '19'], [13.23, '14'], [9.41, '12'], [11.46, '17'], [7.17, '04'], [8.13, '00'], [11.38, '01'], [5.58, '09'], [13.44, '10'], [7.99, '23'], [10.25, '08'], [16.8, '16'], [7.85, '07'], [21.52, '20'], [6.75, '22'], [14.74, '13'], [38.59, '15'], [13.2, '18'], [7.8, '03']]


[[38.59, '15'], [23.81, '02'], [21.52, '20'], [16.8, '16'], [16.01, '21'], [14.74, '13'], [13.44, '10'], [13.23, '14'], [13.2, '18'], [11.46, '17'], [11.38, '01'], [11.05, '11'], [10.8, '19'], [10.25, '08'], [10.09, '05'], [9.41, '12'], [9.02, '06'], [8.13, '00'], [7.99, '23'], [7.85, '07'], [7.8, '03'], [7.17, '04'], [6.75, '22'], [5.58, '09']]




In [58]:
### Formalize the swap in a sexy new set name ###
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print("Top 5 Hours for Ask Posts Comments")

for pair in sorted_swap[:5]:
    avg_comments=pair[0]
    hour=pair[1]
    hour=dt.datetime.strptime(hour,"%H")
    hour=dt.datetime.strftime(hour,"%H")
    hour=hour+":00"
    output="{hour}: {avg:.2f} average comments per post".format(hour=hour,avg=avg_comments)
    print(output)


Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


## Section 3: Conclusion and Recommendations

Late afternoon and late evening time are ideal for receiving high comment numbers. Specifically 3-4pm seems to be strong. This may due to it being near the end of work and posts at that time will receive a bit of traction during the hour before work ends and get a huge boost from people finishing work. There often may be a significant lag from positing to when the comments come in. And so sometime, like as we have seen here, posting at 5pm when people get off work may not be as good of an idea as one would think. But rather to preemt close of work by an hour or two. 

### Further Analysis
There is a lot more we could do with this dataset.
Here are some next steps for you to consider:

1. Determine if show or ask posts receive more points on average.
2. Determine if posts created at a certain time are more likely to receive more points.
3. Compare your results to the average number of comments and points other posts receive.
4. Use Dataquest's data science project style guide to format your project.